<a href="https://colab.research.google.com/github/Dutra-Apex/llm-joc/blob/main/pre-processing/Cosine_similarity_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 1.8 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer, util

In [3]:
# Load a pre-trained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
#TODO make a list of all the sentences

with open("/content/1-7-basics-bootcamp.txt", "r") as file:
  text_list=[]
  for line in file:
    text_list.append(line.strip())


In [12]:
discord_sentences = text_list

In [31]:
print(len(discord_sentences))

17231


In [34]:
discord_df = pd.DataFrame(discord_sentences)

In [35]:
unique_discord = discord_df.drop_duplicates()

In [36]:
print(unique_discord.shape)

(4303, 1)


In [11]:
#TODO make a list of comparison sentences
# comparison_sentences = ["How do i access the courses?",
#                         "How do I use turtle to go right left forward pendown to make triangles and squares?"
#                         "What is the syntax for declaring a function?",
#                         "What is the difference between for loops and while loops?"]

In [38]:
comparison_sentences = ["How do i access the courses?",
                        "Where do i go for JOC video recordings?",
                        "Where can i find the private YouTube channel?",
                        "Which llm are we using for the data science project?",
                        "Can you explain what the daily bot is?",
                        "whats the easiest way to start learning about CI/CD?"]


In [13]:
embeddings_discord = model.encode(discord_sentences)


In [39]:
embeddings_comparison = model.encode(comparison_sentences)

In [40]:
similarities = util.cos_sim(embeddings_discord, embeddings_comparison)

In [15]:
import pandas as pd

In [41]:
df = pd.DataFrame(discord_sentences, columns = ["discord_post"])

In [42]:
# Add new columns with similarity scores
for i, comparison_sentence in enumerate(comparison_sentences):
    df[f"similarity_{i+1}"] = similarities[:, i]


In [43]:
# Print the DataFrame with cosine similarity scores
print(df.head().to_string(index=False))

                                                                                                                                                                                                                                 discord_post  similarity_1  similarity_2  similarity_3  similarity_4  similarity_5  similarity_6
                                         Q: I keep running into this where it looks like there is a video or a walkthrough and nothing appears. Do other people have this problem and is this part of the curriculum? <@&1156625309366485052>      0.307883      0.110684      0.073151      0.080978      0.145448      0.163097
                                                                                                                                               A: I have this happen frequently, 98% of the time if I just refresh the page the video pops up      0.001533      0.137191      0.180495     -0.047021      0.153959     -0.061873
                                  

In [44]:
sim_columns = df.columns[1:]

In [45]:
df["highest sim"] = df[sim_columns].max(axis=1)

In [46]:
sorted_df = df.sort_values(by="highest sim", axis = 0, ascending=False)

In [47]:
print(sorted_df.head())

                                            discord_post  similarity_1  \
16000  Q: HI is anyone else having trouble accessing ...      0.704959   
8403   Q: HI is anyone else having trouble accessing ...      0.704959   
7899   Q: im confused, my courses page is empty for m...      0.672468   
15496  Q: im confused, my courses page is empty for m...      0.672468   
1343   Q: Hey Donte! Have you tried logging in here? ...      0.596890   

       similarity_2  similarity_3  similarity_4  similarity_5  similarity_6  \
16000      0.075882      0.034209      0.105659      0.018940      0.333059   
8403       0.075882      0.034209      0.105659      0.018940      0.333059   
7899       0.106683      0.081383      0.093491      0.068622      0.247706   
15496      0.106683      0.081383      0.093491      0.068622      0.247706   
1343       0.326873      0.087590      0.130914     -0.025822      0.366373   

       highest sim  
16000     0.704959  
8403      0.704959  
7899      0.67246

In [25]:
sorted_df.to_csv("rated_discord_cosine.csv")

In [48]:
print(sorted_df.shape)

(17231, 8)


In [49]:
print(sorted_df["discord_post"][:20].to_string(index=False))

Q: HI is anyone else having trouble accessing c...
Q: HI is anyone else having trouble accessing c...
Q: im confused, my courses page is empty for me...
Q: im confused, my courses page is empty for me...
Q: Hey Donte! Have you tried logging in here? h...
Q: Hey Donte! Have you tried logging in here? h...
Q: Hey Donte! Have you tried logging in here? h...
A: That’s fine - click on the link in this mess...
A: That’s fine - click on the link in this mess...
A: Hey Donte! Have you tried logging in here? h...
A: Hey Donte! Have you tried logging in here? h...
A: Hey Donte! Have you tried logging in here? h...
A: Hello <@1119392015189282877> - there is no J...
A: Hello <@1119392015189282877> - there is no J...
A: Hello <@1119392015189282877> - there is no J...
A: Great - Most of the updates have happened be...
A: Great - Most of the updates have happened be...
A: Great - Most of the updates have happened be...
Q: The whole course is very “go at your own pac...
Q: The whole course is very “go

In [51]:
unique_df = sorted_df.drop_duplicates(subset='discord_post')  # Keep first occurrences of unique values in 'col1'
print("Using drop_duplicates:\n", unique_df)

Using drop_duplicates:
                                             discord_post  similarity_1  \
16000  Q: HI is anyone else having trouble accessing ...      0.704959   
7899   Q: im confused, my courses page is empty for m...      0.672468   
1343   Q: Hey Donte! Have you tried logging in here? ...      0.596890   
7750   A: That’s fine - click on the link in this mes...      0.588463   
10981  A: Hey Donte! Have you tried logging in here? ...      0.566845   
...                                                  ...           ...   
9475   A: Ah ok I thought so on that but wasn’t too c...     -0.070464   
12768  A: At the moment, it's only making the charact...     -0.013435   
12495  A: Not quite - it might feel less dry than tha...     -0.129857   
12491  Q: Not quite - it might feel less dry than tha...     -0.133459   
6572                        A: That's not a slight flaw!     -0.126081   

       similarity_2  similarity_3  similarity_4  similarity_5  similarity_6  \
16000   

In [53]:
unique_df.to_csv("unique_rated_discord_cosine2.csv")


In [52]:
print(unique_df.shape)

(4303, 8)
